In [24]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [25]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
prediction_fn = interpreter.get_signature_runner('serving_default')

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [26]:
print(input_details)
print(output_details)

[{'name': 'serving_default_inputs:0', 'index': 0, 'shape': array([  1, 543,   3], dtype=int32), 'shape_signature': array([ -1, 543,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 1672, 'shape': array([250], dtype=int32), 'shape_signature': array([250], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [27]:
pq_path = "data/2044/3127189.parquet"
ROWS_PER_FRAME = 543

In [28]:
data_columns = ['x', 'y', 'z']
data = pd.read_parquet(pq_path, columns=data_columns) # change the code 
n_frames = int(len(data) / ROWS_PER_FRAME)
data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
xyz = data.astype(np.float32)

In [29]:
from multiprocessing import cpu_count
def read_json_file(file_path):
    """Read a JSON file and parse it into a Python object.

    Args:
        file_path (str): The path to the JSON file to read.

    Returns:
        dict: A dictionary object representing the JSON data.
        
    Raises:
        FileNotFoundError: If the specified file path does not exist.
        ValueError: If the specified file path does not contain valid JSON data.
    """
    try:
        # Open the file and load the JSON data into a Python object
        with open(file_path, 'r') as file:
            json_data = json.load(file)
        return json_data
    except FileNotFoundError:
        # Raise an error if the file path does not exist
        raise FileNotFoundError(f"File not found: {file_path}")
    except ValueError:
        # Raise an error if the file does not contain valid JSON data
        raise ValueError(f"Invalid JSON data in file: {file_path}")

cpu_count()

4

In [30]:
import os
import json
train_df = pd.read_csv('train.csv')
print("\n\n... LOAD SIGN TO PREDICTION INDEX MAP FROM JSON FILE ...\n")
s2p_map = {k.lower():v for k,v in read_json_file(os.path.join("sign_to_prediction_index_map.json")).items()}
p2s_map = {v:k for k,v in read_json_file(os.path.join("sign_to_prediction_index_map.json")).items()}
encoder = lambda x: s2p_map.get(x.lower())
decoder = lambda x: p2s_map.get(x)
# print(s2p_map)
train_df['label'] = train_df.sign.map(encoder)



... LOAD SIGN TO PREDICTION INDEX MAP FROM JSON FILE ...



In [31]:

output = prediction_fn(inputs=xyz)
p = output['outputs'].reshape(-1)
print(p)
decoder(p.argmax())


[-8.86820793e-01  6.81655049e-01  9.37719643e-01  6.66603386e-01
 -4.61473018e-02  3.79723966e-01 -1.14137733e+00  9.99376476e-01
  1.71679929e-01  1.25748551e+00 -9.05560888e-03 -5.19064248e-01
 -1.50597262e+00 -5.48428535e-01 -1.36578274e+00 -5.57539940e-01
 -1.04789793e+00  2.34915629e-01 -3.73249382e-01 -9.08312798e-01
 -1.67654896e+00 -8.54372084e-01 -1.27558935e+00 -7.37284124e-01
  1.40889657e+00 -1.64499998e+00  1.11103572e-01 -6.17511749e-01
 -4.91352260e-01 -1.56341648e+00 -7.31140554e-01 -7.82747269e-01
 -8.09972659e-02 -5.32567799e-01 -8.34588647e-01  2.97168851e-01
 -1.01388074e-01  5.86076617e-01  6.99312806e-01  3.24538618e-01
 -2.72002131e-01  3.68671227e+00 -5.39129853e-01  2.15459913e-01
 -1.00367177e+00 -3.93847197e-01 -4.07405943e-01 -3.28699350e-01
 -5.63031316e-01  7.05047548e-01 -6.60112143e-01 -1.08669126e+00
  2.72267866e+00  2.18810415e+00 -2.19982195e+00  9.81597006e-02
  1.02145247e-01 -8.17297220e-01  1.64145172e-01 -1.09884191e+00
 -9.66374755e-01 -2.70656

'eye'

In [ ]:
# get the frame data from each parquet file in the 20444